In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
df = pd.read_csv('demand1 (1).csv')
# Pivot the data for multi-time series dataset
df_pivot = df.pivot_table(values='demand', index='timestamp', columns=['region', 'city', 'item_id'])

In [ ]:
df

,timestamp,region,city,item_id,demand,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,1-2-2018,region_1,city_2,item_1,148,NaN,NaN,NaN,NaN,NaN
1,1-3-2018,region_1,city_2,item_1,124,NaN,NaN,NaN,NaN,NaN
2,1-4-2018,region_1,city_2,item_1,243,NaN,NaN,NaN,NaN,NaN
3,1-5-2018,region_1,city_2,item_1,199,NaN,NaN,NaN,NaN,NaN
4,1-6-2018,region_1,city_2,item_1,100,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
13459,5-1-2024,region_2,city_1,item_2,2986,NaN,NaN,NaN,NaN,NaN
13460,5-2-2024,region_2,city_1,item_2,2742,NaN,NaN,NaN,NaN,NaN
13461,5-3-2024,region_2,city_1,item_2,1722,NaN,NaN,NaN,NaN,NaN
13462,5-4-2024,region_2,city_1,item_2,542,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Fill missing values
df_pivot = df_pivot.fillna(0)

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_pivot)

In [ ]:
# Prepare the data for LSTM (assuming a look-back window of 7 days)
look_back = 7
X, y = [], []
for i in range(look_back, len(df_scaled)):
    X.append(df_scaled[i-look_back:i])
    y.append(df_scaled[i])

X, y = np.array(X), np.array(y)

In [ ]:
# Reshape X for LSTM (samples, time steps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

In [ ]:
# Split the data into train and test sets
train_size = int(X.shape[0] * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1]))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - loss: 0.1141 - val_loss: 0.0506
Epoch 2/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0562 - val_loss: 0.0498
Epoch 3/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0541 - val_loss: 0.0489
Epoch 4/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0507 - val_loss: 0.0471
Epoch 5/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0491 - val_loss: 0.0443
Epoch 6/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0450 - val_loss: 0.0418
Epoch 7/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0429 - val_loss: 0.0415
Epoch 8/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0424 - val_loss: 0.0439
Epoch 9/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0406 - val_loss: 0.0373
Epoch 10/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0383 - val_loss: 0.0364
Epoch 11/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0387 - val_loss: 0.0357
Epoch 12/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0379 - val_loss: 0.0

In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)

# Print a sample forecast
print("Sample Forecast:", y_pred[:7])

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
Sample Forecast: [[ 111.63178   443.22107  1949.1443    580.04456  3488.7986    202.43373 ]
 [ 144.98555   597.5      2737.3176   1109.4326   6575.8228    195.81114 ]
 [ 238.27377  1033.8405   2894.8794    916.54846  6109.8223    191.85979 ]
 [ 241.00528  1132.6287   2891.446     575.472    4385.9917    151.46509 ]
 [  81.082375  434.46234  2209.2317    112.64945   795.00916   112.418976]
 [  33.102013  164.29161  2213.2043    -11.659644   21.536934  189.49167 ]
 [ 169.34166   656.0045   1491.1163   1125.2694   6468.5474    187.13133 ]]


In [28]:
# Ensuring df_pivot is sorted by date
df_pivot = df_pivot.sort_index()

In [29]:
last_sequence = df_scaled[-look_back:]  # Last sequence from the original scaled data

In [30]:
# Forecast next 7 days
forecast = []
input_sequence = last_sequence

for _ in range(7):  # For 7 days forecast
    next_day_prediction = model.predict(input_sequence[np.newaxis, :, :])
    forecast.append(next_day_prediction.flatten())  # Flatten and append the result
    # Update the input sequence by adding the new prediction and removing the first day
    input_sequence = np.append(input_sequence[1:], next_day_prediction, axis=0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [31]:
# Forecast the next 14 days (continuing from the 7-day forecast)
forecast_14 = forecast.copy()

for _ in range(7):  # For additional 7 days to make 14 days total
    next_day_prediction = model.predict(input_sequence[np.newaxis, :, :])
    forecast_14.append(next_day_prediction.flatten())  # Flatten and append the result
    # Update the input sequence by adding the new prediction and removing the first day
    input_sequence = np.append(input_sequence[1:], next_day_prediction, axis=0)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [32]:
# Convert forecasts back to original scale
forecast_7_days = scaler.inverse_transform(np.array(forecast))
forecast_14_days = scaler.inverse_transform(np.array(forecast_14))

In [ ]:
# Explicitly set the last date to 5-5-2024 as it can misidentify and give forecast for dates already present
last_date = pd.to_datetime('2024-05-05')

In [33]:
# Prepare date range for the forecasts starting from the day after the last date
forecast_dates_7 = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=7, freq='D')
forecast_dates_14 = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=14, freq='D')

In [39]:
# Sum the predictions across all regions, cities, and items to get total demand
total_demand_7 = forecast_7_days.sum(axis=1)
total_demand_14 = forecast_14_days.sum(axis=1)


In [40]:
forecast_7_df = pd.DataFrame({'Date': forecast_dates_7, 'Total Demand': total_demand_7})
forecast_14_df = pd.DataFrame({'Date': forecast_dates_14, 'Total Demand': total_demand_14})

In [41]:
# Print the forecasts
print("7-Day Forecast:")
print(forecast_7_df)

print("\n14-Day Forecast:")
print(forecast_14_df)


7-Day Forecast:
        Date  Total Demand
0 2024-05-06   8743.716797
1 2024-05-07  12454.108398
2 2024-05-08  11852.308594
3 2024-05-09  10533.844727
4 2024-05-10   5472.631348
5 2024-05-11   3808.962402
6 2024-05-12  10798.052734

14-Day Forecast:
         Date  Total Demand
0  2024-05-06   8743.716797
1  2024-05-07  12454.108398
2  2024-05-08  11852.308594
3  2024-05-09  10533.844727
4  2024-05-10   5472.631348
5  2024-05-11   3808.962402
6  2024-05-12  10798.052734
7  2024-05-13  11550.448242
8  2024-05-14  10472.751953
9  2024-05-15   8160.315430
10 2024-05-16   4486.854492
11 2024-05-17   6671.732422
12 2024-05-18  10452.154297
13 2024-05-19  10480.199219
